In [82]:
import numpy as np
import random
import cv2
class AntColonySegmentation:
    def __init__(self, image, num_ants, max_iterations, alpha, beta, rho):
        self.image = image
        self.num_ants = num_ants
        self.max_iterations = max_iterations
        self.alpha = alpha  # pheromone importance
        self.beta = beta  # visibility (intensity) importance
        self.rho = rho  # pheromone evaporation rate

        self.pheromones = np.ones([self.image.shape[0],self.image.shape[1]])

    def initialize_ants(self):
        ants = []
        for _ in range(self.num_ants):
            ant = {'path': [], 'intensity_sum': 0.0}
            ants.append(ant)
        return ants

    def construct_segment(self, ant):
        #current_position = np.random.randint(0, len(self.image), dtype=int)
        current_position = (np.random.randint(0, self.image.shape[0]), np.random.randint(0, self.image.shape[1]))
        ant['path'].append(current_position)

        for _ in range(10000):
            neighbors = self.get_neighbors(current_position)
            probabilities = self.calculate_probabilities(current_position, neighbors, ant['path'])

            #flat_neighbors = np.array(neighbors).ravel()
            #print(len(neighbors),probabilities.shape)
            pom = [_ for _ in range(len(neighbors))]
            next_position = np.random.choice(pom, p=probabilities)
            next_position = neighbors[next_position]
            # while 1:
            #     next_position = np.random.choice(pom, p=probabilities)
            #     next_position = neighbors[next_position]
            #     if next_position in ant['path']:
            #         continue
            #     else:
            #         break
            ant['path'].append(next_position)
            current_position = next_position

    def get_neighbors(self, position):
        height, width,_ = self.image.shape
        row, col = position
        neighbors = []
        neighbors_offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for offset_row, offset_col in neighbors_offsets:
            new_row, new_col = row + offset_row, col + offset_col
            if 0 <= new_row < self.image.shape[0] and 0 <= new_col < self.image.shape[1]:
                neighbors.append((new_row, new_col))

        return neighbors

    def calculate_probabilities(self, current_position, neighbors, path):
        intensity = self.image[current_position][1]
        probabilities = []

        #print(neighbors)
        
        for neighbor in neighbors:
            if neighbor not in path:
                neighbor_intensity = self.image[neighbor][1]
                pheromone = self.pheromones[neighbor]
                # if neighbor_intensity == intensity:
                #     visibility = 1.0
                # else:
                #     #visibility = 1.0 / abs(intensity - neighbor_intensity)
                #     visibility = 1.0 / np.abs(intensity-neighbor_intensity)
                if neighbor_intensity>intensity:
                    visibility = 0.95
                else:
                    visibility = 0.05
                probability = (pheromone ** self.alpha) * (visibility ** self.beta)
                probabilities.append(probability)
                #print(pheromone.shape,probability.shape)
            else:
                probabilities.append(0.0)

        probabilities = np.array(probabilities)
        if np.any(np.isnan(probabilities)):
            probabilities[np.isnan(probabilities)] = 0
        if np.any(np.isinf(probabilities)):
            probabilities[np.isinf(probabilities)] = 1
        total_probability = sum(probabilities)
        if total_probability > 0:
            probabilities /= total_probability
        else:
            # All neighbors are already in the path; choose randomly
            probabilities = np.ones(len(neighbors)) / len(neighbors)
            #print(probabilities,neighbors)
        #if np.any(np.isnan(probabilities)):
        #probabilities[np.isnan(probabilities)] = 0
        
        return probabilities

    def update_pheromones(self, ants):
        self.pheromones *= (1 - self.rho)  # Evaporation
        for ant in ants:
            for position in ant['path']:
                self.pheromones[position] += 1.0 / ant['intensity_sum']

    def run(self):
        ants = self.initialize_ants()
        for iteration in range(self.max_iterations):
            #ants = self.initialize_ants()

            for ant in ants:
                self.construct_segment(ant)
                ant['intensity_sum'] = sum(self.image[position][1] for position in ant['path'])
                # for position in ant['path']:
                #     print(self.image[position],end=' ')
                # print('\n')

            self.update_pheromones(ants)
            

        # Choose the best segment (ant) based on intensity sum
        best_ant = max(ants, key=lambda ant: ant['intensity_sum'])
        segmentation_result = np.zeros_like(self.image)
        #segmentation_result.ravel()[best_ant['path']] = 255
        # print('best path size',len(best_ant['path']))
        # for position in best_ant['path']:
        #     print(self.image[position],end=' ')
        # print('\n')
        for position in best_ant['path']:
            segmentation_result[position] = self.image[position]

        return segmentation_result


In [83]:
image = cv2.imread("img/8s.png")

num_ants = 5
max_iterations = 2

alpha = 0.8
beta = 1.3
rho = 0.1

# Create AntColonySegmentation object
aco_segmentation = AntColonySegmentation(image, num_ants, max_iterations, alpha, beta, rho)

segmented_image = aco_segmentation.run()

# Display the original and segmented images
#cv2.imshow("Original Image", image)
cv2.imshow("Segmented Image", segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [77]:
image = cv2.imread("img/2.jpg")
#image = cv2.imread("juve.jpg", cv2.IMREAD_GRAYSCALE)
p=np.ones_like(image, dtype=float)
b= np.ones([2,4])
print(b)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [2]:
import cv2
import numpy as np

# Load an example image
image_path = '8.jpg'
original_image = cv2.imread(image_path)

# Convert the image to grayscale
gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

# Apply a threshold to create a binary image
_, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)

# Find contours in the binary image
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a black image as a canvas
segmented_image = np.zeros_like(original_image)

# Draw contours on the segmented image
cv2.drawContours(segmented_image, contours, -1, (0, 255, 0), 2)

# Display the original and segmented images
cv2.imshow('Original Image', original_image)
cv2.imshow('Segmented Image', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
